# Example for cleaning an existing dataset and creating a table schema

- Inspecting fields
- Setting consistent column names
- Setting correct data types
- Writing the table schema
- Creating the data package/directory structure

In [1]:
import pandas as pd

# Data inspecting/cleaning

In [5]:
DATA = pd.read_csv('PMCLs_00-18.csv')
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34510 entries, 0 to 34509
Data columns (total 18 columns):
STATION_ID            34510 non-null int64
STATION_NAME          34510 non-null object
FULL_STATION_NAME     34510 non-null object
STATION_NUMBER        34510 non-null object
STATION_TYPE          34510 non-null object
LATITUDE              32614 non-null float64
LONGITUDE             32614 non-null float64
COUNTY_NAME           34510 non-null object
SAMPLE_CODE           34510 non-null object
SAMPLE_DATE           34510 non-null object
SAMPLE_DEPTH          34311 non-null float64
SAMPLE_DEPTH_UNITS    34510 non-null object
PARAMETER             34510 non-null object
RESULT                33481 non-null float64
REPORTING_LIMIT       34509 non-null float64
UNITS                 34510 non-null object
METHOD_NAME           34510 non-null object
sampleYear            34510 non-null int64
dtypes: float64(5), int64(2), object(11)
memory usage: 4.7+ MB


## Column/field names

From `info()` we see that the column names are in good shape. 

The only things to do are:
- Convert to lowercase
- Use consistent casing

This can be done either programmatically or manually, or with a mix of the two.

In [54]:
COLUMN_NAMES_MANUAL = {
    'FULL_STATION_NAME': 'station_name_full',
    'sampleYear': 'sample_year',
}

def get_fixed_name(col):
    # if the input column is not in the dictionary representing manual assignment, simply use str.lower()
    return COLUMN_NAMES_MANUAL.get(col, col.lower())

In [20]:
DATA = DATA.rename(columns=get_fixed_name)
DATA.columns

Index(['station_id', 'station_name', 'station_name_full', 'station_number',
       'station_type', 'latitude', 'longitude', 'county_name', 'sample_code',
       'sample_date', 'sample_depth', 'sample_depth_units', 'parameter',
       'result', 'reporting_limit', 'units', 'method_name', 'sample_year'],
      dtype='object')

## Categorical fields

- Categorical fields are fields that can have only a limited set of values
    - Equivalent to "enum" types in most programming languages
- They are expressed in pandas with the `Categorical` dtype
- If categorical dtypes are set on a DataFrame column, the corresponding field will have a `constraints.enum` property set wit the correct values when generating the table schema from the DataFrame object

The decision whether a field should be interpreted as categorical or not is sometimes not easy. Inspecting the number of unique values present in the data is a good first criterion:

In [21]:
DATA.apply(lambda s: s.nunique())

station_id             1101
station_name           1099
station_name_full      1092
station_number         1101
station_type              3
latitude                848
longitude               726
county_name              37
sample_code           15748
sample_date           13895
sample_depth            119
sample_depth_units        2
parameter                 5
result                 2566
reporting_limit         113
units                     3
method_name               8
sample_year              19
dtype: int64


In this case, `station_type`, `sample_depth_units`, `parameter`, `units` and `method_name` all have below 10 values, so it makes sense to consider them as categorical. 
`sample_year` has low number of values, but it doesn't really represent a category.
It could be argued that `county_name` should be categorical, but since we know that there are other counties not present in this dataset, we can leave it as it is.

We can inspect the individual values for the categorical candidates to confirm:

In [53]:
def inspect_values(df, cols):
    separator = '-' * 10
    for col in cols:
        print(separator)
        display(df[col].value_counts())


DATA.pipe(inspect_values, ['station_type', 'sample_depth_units', 'parameter', 'units', 'method_name'])

----------


Surface Water    28975
Groundwater       3608
Other             1927
Name: station_type, dtype: int64

----------


Meters    32814
Feet       1696
Name: sample_depth_units, dtype: int64

----------


Dissolved Nitrate    10264
Total Arsenic         7747
Total Lead            7423
Total Selenium        7237
Total Mercury         1839
Name: parameter, dtype: int64

----------


ug/L    18232
mg/L    14575
ng/L     1703
Name: units, dtype: int64

----------


EPA 1638 (T)            18232
EPA 300.0 28d Hold       9089
EPA 200.8 (T)            4156
EPA 1631 E (T)           1703
EPA 300.0 48 hr          1175
EPA 245.1(T)              135
EPA 200.8 (D)              19
EPA 200.8 (Hg Total)        1
Name: method_name, dtype: int64

In [24]:
def set_as_categorical(df, cols):
    return df.astype({col: 'category' for col in cols})

DATA = DATA.pipe(set_as_categorical, ['station_type', 'sample_depth_units', 'parameter', 'units', 'method_name'])
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34510 entries, 0 to 34509
Data columns (total 18 columns):
station_id            34510 non-null int64
station_name          34510 non-null object
station_name_full     34510 non-null object
station_number        34510 non-null object
station_type          34510 non-null category
latitude              32614 non-null float64
longitude             32614 non-null float64
county_name           34510 non-null object
sample_code           34510 non-null object
sample_date           34510 non-null object
sample_depth          34311 non-null float64
sample_depth_units    34510 non-null category
parameter             34510 non-null category
result                33481 non-null float64
reporting_limit       34509 non-null float64
units                 34510 non-null category
method_name           34510 non-null category
sample_year           34510 non-null int64
dtypes: category(5), float64(5), int64(2), object(6)
memory usage: 3.6+ MB


## Datetime fields

As with categorical data, using the native pandas datetime dtypes is not strictly necessary for the purpose of building the table schema, but helps with validation (making sure that the output format is consistent), and we'll have the correct type in the table schema field.

The `to_datetime()` function in the main pandas module should be robust enough to convert automatically from many input formats.

In [43]:
DATA = (DATA
        .assign(sample_date=lambda d: pd.to_datetime(d.sample_date))
)
# this should give the same result as
# DATA['sample_year'] = pd.to_datetime(DATA['sample_year'])
# but is somewhat more robust
DATA.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 34510 entries, 0 to 34509
Data columns (total 18 columns):
station_id            34510 non-null int64
station_name          34510 non-null object
station_name_full     34510 non-null object
station_number        34510 non-null object
station_type          34510 non-null category
latitude              32614 non-null float64
longitude             32614 non-null float64
county_name           34510 non-null object
sample_code           34510 non-null object
sample_date           34510 non-null datetime64[ns]
sample_depth          34311 non-null float64
sample_depth_units    34510 non-null category
parameter             34510 non-null category
result                33481 non-null float64
reporting_limit       34509 non-null float64
units                 34510 non-null category
method_name           34510 non-null category
sample_year           34510 non-null int64
dtypes: category(5), datetime64[ns](1), float64(5), int64(2), object(5)
memory

We see that the `sample_year` column matches exactly the year in `sample_date`. In principle, then, `sample_year` is redundant, and could be dropped.

In [42]:
DATA[lambda d: d.sample_date.dt.year != d.sample_year]

,station_id,station_name,station_name_full,station_number,station_type,latitude,longitude,county_name,sample_code,sample_date,sample_depth,sample_depth_units,parameter,result,reporting_limit,units,method_name,sample_year


# Table schema

## Creating table schema from the DataFrame

The `build_table_schema()` function allows to get a 

Few things to note:
- The output of `build_table_schema()` is a Python object (dictionary)
    - It can be modified (either manually or programmatically) from within the notebook
- It must be converted to a string (for copy-pasting) or saved to a file to be used with other applications
    - Use the `json` module of the standard library for this


In [52]:
from pandas.io.json import build_table_schema
import json

def get_table_schema(df):
    return build_table_schema(df, index=False, version=False)

json.dumps(DATA.pipe(get_table_schema))

'{"fields": [{"name": "station_id", "type": "integer"}, {"name": "station_name", "type": "string"}, {"name": "station_name_full", "type": "string"}, {"name": "station_number", "type": "string"}, {"name": "station_type", "type": "any", "constraints": {"enum": ["Groundwater", "Other", "Surface Water"]}, "ordered": false}, {"name": "latitude", "type": "number"}, {"name": "longitude", "type": "number"}, {"name": "county_name", "type": "string"}, {"name": "sample_code", "type": "string"}, {"name": "sample_date", "type": "datetime"}, {"name": "sample_depth", "type": "number"}, {"name": "sample_depth_units", "type": "any", "constraints": {"enum": ["Feet", "Meters"]}, "ordered": false}, {"name": "parameter", "type": "any", "constraints": {"enum": ["Dissolved Nitrate", "Total Arsenic", "Total Lead", "Total Mercury", "Total Selenium"]}, "ordered": false}, {"name": "result", "type": "number"}, {"name": "reporting_limit", "type": "number"}, {"name": "units", "type": "any", "constraints": {"enum": 

## Editing the table schema

Although this method works reasonably well, some parts of the table schema cannot be inferred from the existing data and must be filed in manually.  
The most relevant properties are:
- `title`: the human-readable counterpart to `name` (what one would e.g. use for an axis label in a figure, or on the header of a table in a published format)
- `description`: the data dictionary entry for the field. It can contain both data knowledge (e.g. additional properties of the data), and, most importantly, domain knowledge, in the form of metadata, specifications, sources, and so on

Unlike `name`, these metadata fields of the table schema are not mandatory, but having them filled in the schema, since it allows to concentrate the information in one place (instead of e.g. maintaining a separate PDF file for the data dictionary).

JSON files are somewhat annoying to edit directly. A few options to make things easier:
- Using a code editor with good JSON support, such as VSCode
- Using a dedicated JSON editor (online or offline) such as https://jsoneditoronline.org/

Once the schema is filled in, it can be incorporated within the `datapackage.json` file.

----
# Bonus: Normalizing tables

In [33]:
def get_normalized(df, pk=None, **kwargs):
    return df.drop_duplicates(subset=pk, **kwargs)

(get_normalized(DATA[[
    'station_id', 'station_name', 'station_name_full', 'station_number', 'station_type',
    'latitude', 'longitude', 'county_name',
    ]],
    pk='station_id')
 .set_index('station_id').sort_index()
)

,station_name,station_name_full,station_number,station_type,latitude,longitude,county_name
station_id,,,,,,,
1,AMERICAN,American River at Water Treatment Plant,A0714010,Surface Water,38.5596,-121.417,Sacramento
10,CLIFTON,Clifton Court Intake,KA000000,Surface Water,37.8298,-121.557,Contra Costa
12,H.O. Banks Headworks,Delta P.P. Headworks at H.O. Banks PP,KA000331,Surface Water,37.8019,-121.620,Alameda
73,COLUSA,Ag Drain on Colusa Basin Main Drain,A0294500,Surface Water,38.7993,-121.725,Yolo
78,TWITCHELLPP01,"Ag Drain on Twitchell Isl., PP. No. 1",B9V80661391,Surface Water,38.0966,-121.651,Sacramento
80,CHECK 13,"CA Aqueduct, Ck 13, O'Neill Outlet",KA007089,Surface Water,37.0742,-121.015,Merced
87,Del Valle Reservoir,Lake Del Valle at Glory Hole,DV001000,Surface Water,37.6147,-121.746,Alameda
99,OR nr Byron Hwy 4 Br,Old R. nr. Byron (St 9) (NEAR HWY 4 BRIDGE),B9D75351342,Surface Water,37.8910,-121.570,Contra Costa
114,SACWSACINT,Sacramento River at W. Sac Intake Structure,A0210451,Surface Water,38.5980,-121.549,Yolo
